<h3><b>Author : Sapnil Patel<b></h3>
<h3><b>Date : 22/12/2023<b></h3>

In [1]:
import os
from dotenv import load_dotenv
from langchain.llms import GooglePalm

In [2]:
# import environment variables from .env files
load_dotenv()

True

In [3]:
api_key=os.environ['api_key']

In [4]:

llm = GooglePalm(google_api_key=api_key,temperature=0.2)

c:\Users\SAPNIL\Desktop\Learnings\LLM_SQL\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
print(llm("Write me an poem of Fafda"))

**Fafda**

Fafda, fafda, oh so crispy,
A Gujarati snack that's so delish.
With its chickpea flour base,
And its spices that make your taste buds dance,
Fafda is a must-try for all.

Whether you eat it with chutney,
Or with just a squeeze of lemon,
Fafda is always a crowd-pleaser.
So next time you're looking for a quick and easy snack,
Reach for a fafda and enjoy!

Fafda, fafda, oh so delicious,
A taste of Gujarat that's sure to please.
So next time you're in the mood for something special,
Don't forget to fafda up!


In [6]:
# Let's connect vs code with database with pymysql library
from langchain.utilities import SQLDatabase

db_user = "root"
db_password = "root"
db_host = "localhost"
db_name = "sapnil_tshirts"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)

In [7]:
# Check if we have successfully connected or not
print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [8]:
from langchain_experimental.sql import SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
qns1 = db_chain("How many total nike white tshirts are left?")



> Entering new SQLDatabaseChain chain...
How many total nike white tshirts are left?
SQLQuery:SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White'
SQLResult: [(Decimal('250'),)]
Answer:250
> Finished chain.


In [9]:
qns2 = db_chain.run("How much is the price of the inventory for all small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT SUM(price) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('322'),)]
Answer:322
> Finished chain.


In [10]:
qns2 = db_chain.run("SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLQuery:SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('18551'),)]
Answer:18551
> Finished chain.


In [11]:
# qns3 = db_chain.run("If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?" )

In [12]:
qns3 = db_chain.run("""SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """)



> Entering new SQLDatabaseChain chain...
SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 
SQLQuery:SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('27880.100000'),)]
Answer:27880.1
> Finished chain.


In [13]:
qns4 = db_chain.run("How many dollars will I get if I sell all the tshirts of Levi")



> Entering new SQLDatabaseChain chain...
How many dollars will I get if I sell all the tshirts of Levi
SQLQuery:SELECT SUM(price) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('437'),)]
Answer:437
> Finished chain.


In [14]:
qns4 = db_chain.run("SELECT SUM(price*stock_quantity) FROM t_shirts WHERE brand = 'Levi")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price*stock_quantity) FROM t_shirts WHERE brand = 'Levi
SQLQuery:SELECT SUM(price*stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('28334'),)]
Answer:28334
> Finished chain.


In [15]:
qns5 = db_chain.run("How many white color Levi's t-shirts we have available?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's t-shirts we have available?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(89,), (90,), (20,), (66,)]
Answer:89
> Finished chain.


In [16]:
qns5 = db_chain.run("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")



> Entering new SQLDatabaseChain chain...
SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('265'),)]
Answer:265
> Finished chain.


<h4><b>Few Shot Learning<b></h3>

In [64]:
few_shots = [
    {'Question' : "How many total nike white tshirts are left?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM sapnil_tshirts.t_shirts WHERE brand='Nike' and color='White';",
     'SQLResult': "Result of the SQL query",
     'Answer' : "250"},
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': "18551"},
    {
    {'Question': "how much is the price of the inventory for all small size t-shirts?",
     'SQLQuery':"SELECT sum(price*stock_quantity) FROM sapnil_tshirts.t_shirts WHERE size='S';",
     'SQLResult': "Result of the SQL query",
     'Answer': "18551"},
    }
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post      discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
      (select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
      group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
      """,
     'SQLResult': "Result of the SQL query",
     'Answer': "27880.1"} ,
    {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
     'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
     'SQLResult': "Result of the SQL query",
     'Answer' : "28334"},
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : "265"
    }
    
]

SyntaxError: invalid syntax (1650160275.py, line 16)

In [65]:
from langchain.embeddings import HuggingFaceEmbeddings

In [66]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

emb = embeddings.embed_query("How many white color Levi's shirt I have")

In [67]:
emb[:5]

[0.006943074520677328,
 0.048174694180488586,
 0.02245817892253399,
 -0.014070001430809498,
 0.06808178871870041]

In [68]:
# let's generate thr string of al the values of dictionary
to_vectorize = [" ".join(example.values()) for example in few_shots]
to_vectorize[0]

"How many total nike white tshirts are left? SELECT sum(stock_quantity) FROM sapnil_tshirts.t_shirts WHERE brand='Nike' and color='White'; Result of the SQL query 250"

In [69]:
from langchain.vectorstores import Chroma

In [70]:
vectorstore = Chroma.from_texts(texts=to_vectorize,embedding=embeddings,metadatas=few_shots)

In [71]:
from langchain.prompts import  SemanticSimilarityExampleSelector

example_selector = SemanticSimilarityExampleSelector(vectorstore=vectorstore,k=2)

example_selector.select_examples({"Question": "How many Adidas T shirts I have left in my store"})

[{'Answer': '250',
  'Question': 'How many total nike white tshirts are left?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM sapnil_tshirts.t_shirts WHERE brand='Nike' and color='White';",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': '250',
  'Question': 'How many total nike white tshirts are left?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM sapnil_tshirts.t_shirts WHERE brand='Nike' and color='White';",
  'SQLResult': 'Result of the SQL query'}]

* Since our LLM model is making some mistakes like it assumes that there is always start and end dates are given for sales but its not true for every scenario.
*  Another mistake it is doing is that it assumes the price column shows the the price of all the t-shirts and not the price of 1 t-shirt because price column name is "price" and not the "price_per_one_unit". 
* So, let's give it a prompt to reduce the mistakes and increase the accuracy. 
* The prompt is already there in langchain.

In [72]:
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX,_mysql_prompt

In [73]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [74]:
print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [75]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables = ['Question','SQLQuery','SQLResult','Answer'],
    template = "\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}"
)

Fewshot Prompt Template

In [76]:
from langchain.prompts import FewShotPromptTemplate

few_shot_prompt = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt = example_prompt,
    suffix = PROMPT_SUFFIX,
    input_variables = ['input','table_info','top_k'] #These variables are used in the prefix and suffix
)

In [77]:
new_chain = SQLDatabaseChain.from_llm(llm, db, prompt = few_shot_prompt, verbose=True)

In [79]:
new_chain("If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?
SQLQuery:SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
     (select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Nike'
     group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('25300.850000'),)]
Answer:25300.85
> Finished chain.


{'query': 'If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?',
 'result': '25300.85'}